In [1]:
import pandas as pd

In [2]:
# 拆分车型
input_ = pd.read_excel('./Input.xlsx', sheet_name='车型数据', dtype=str)
df_input = input_.copy()
df_input['Vehicle'] = df_input['Vehicle'].apply(lambda x: x.split('\n'))
df_input = df_input.explode('Vehicle').fillna('')
# df_input

In [3]:
# 拆解Make、Model、Year
list_vehicle = df_input['Vehicle'].tolist()
ls_make = []
ls_model = []
ls_years = []
for v in list_vehicle:
    list_split = v.split(' ')
    make = list_split[0]
    years = str(list_split[-1])
    if make in ['Land', 'Aston', 'Alfa']:
        make = ' '.join(list_split[:2])
        model = ' '.join(list_split[2:-1])
    else:
        model = ' '.join(list_split[1:-1])
    ls_make.append(make)
    ls_model.append(model)
    ls_years.append(years)
df_input['Make'] = ls_make
df_input['Model'] = ls_model
df_input['Years'] = ls_years
# df_input

In [4]:
# 获取make_model、start year、end year
df_input['Make Model'] = df_input['Make'] + ' ' + df_input['Model']
df_input['Start Year'] = df_input['Years'].apply(lambda x: x if '-' not in x else x.split('-')[0])
df_input['End Year'] = df_input['Years'].apply(lambda x: x if '-' not in x else x.split('-')[1])
df_input['Start Year'] = df_input['Start Year'].apply(lambda x: x if x in [str(i) for i in range(1950, 2050)] else '9999')
df_input['End Year'] = df_input['End Year'].apply(lambda x: x if x in [str(i) for i in range(1950, 2050)] else '9999')
# df_input

In [5]:
# 特殊车型处理
df_special = pd.read_excel('./Input.xlsx', sheet_name='特殊车型匹配', dtype=str)
df_special['Match_Vehicle'] = df_special['make'] + ' ' + df_special['model']
df_match = pd.merge(df_input, df_special, left_on='Make Model', right_on='Match_Vehicle', how='left')
df_match['make+model'] = df_match['make+model'].fillna(df_match['Make Model'])
df_match = df_match[['NO.', 'Make', 'Model', 'Years', 'make+model', 'Start Year', 'End Year']]
df_match.rename(columns={'make+model': 'Make Model'}, inplace=True)
df_match['Make Model'] = df_match['Make Model'].str.title()
df_match

,NO.,Make,Model,Years,Make Model,Start Year,End Year
0,1,Hyundai,Elantra,2021-2023,Hyundai Elantra,2021,2023
1,2,Hyundai,Sonata,2022-2023,Hyundai Sonata,2022,2023
2,3,Ford,Mustang Mach-E,2021-2024,Ford Mustang Mach-E,2021,2024
3,4,Ford,Mustang Mach-E,2021-2024,Ford Mustang Mach-E,2021,2024
4,5,Jaguar,I-Pace,2018-2024,Jaguar I-Pace,2018,2024
...,...,...,...,...,...,...,...
1352,1353,MINI,Mini,2001-2013,Mini Mini,2001,2013
1353,1354,Audi,A3,2006-2019,Audi A3,2006,2019
1354,1355,Hyundai,Accent III,2005-2010,Hyundai Accent Iii,2005,2010
1355,1356,Audi,A3,2003-2012,Audi A3,2003,2012


In [6]:
# 计算保有量
def sale(df_sale, x, y):
    try:
        result = df_sale.loc[rows.index(x), list(set(y))].values.astype(int).sum()
    except:
        result = 0
    return int(result)
# 匹配保有量
df_sale = pd.read_excel('./Input.xlsx', sheet_name='保有量数据', dtype=str)
df_sale['Make+Model'] = df_sale['Make+Model'].str.title()
rows = df_sale['Make+Model'].tolist()
df_match['columns'] = df_match.apply(lambda row: [i for i in range(int(row['Start Year']), int(row['End Year'])+1)], axis=1)
df_match['保有量'] = df_match.apply(lambda row: sale(df_sale, row['Make Model'], row['columns']), axis=1)
df_match

,NO.,Make,Model,Years,Make Model,Start Year,End Year,columns,保有量
0,1,Hyundai,Elantra,2021-2023,Hyundai Elantra,2021,2023,"[2021, 2022, 2023]",378686
1,2,Hyundai,Sonata,2022-2023,Hyundai Sonata,2022,2023,"[2022, 2023]",92795
2,3,Ford,Mustang Mach-E,2021-2024,Ford Mustang Mach-E,2021,2024,"[2021, 2022, 2023, 2024]",159114
3,4,Ford,Mustang Mach-E,2021-2024,Ford Mustang Mach-E,2021,2024,"[2021, 2022, 2023, 2024]",159114
4,5,Jaguar,I-Pace,2018-2024,Jaguar I-Pace,2018,2024,"[2018, 2019, 2020, 2021, 2022, 2023, 2024]",10182
...,...,...,...,...,...,...,...,...,...
1352,1353,MINI,Mini,2001-2013,Mini Mini,2001,2013,"[2001, 2002, 2003, 2004, 2005, 2006, 2007, 200...",0
1353,1354,Audi,A3,2006-2019,Audi A3,2006,2019,"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",186313
1354,1355,Hyundai,Accent III,2005-2010,Hyundai Accent Iii,2005,2010,"[2005, 2006, 2007, 2008, 2009, 2010]",0
1355,1356,Audi,A3,2003-2012,Audi A3,2003,2012,"[2003, 2004, 2005, 2006, 2007, 2008, 2009, 201...",48740


In [7]:
# 聚合并输出结果
df_sum = df_match.groupby('NO.').sum().reset_index()
df_result = pd.merge(input_, df_sum[['NO.', '保有量']], on='NO.', how='left')
df_result.to_excel('./Output.xlsx', index=False)
df_result

,NO.,Vehicle,保有量
0,1,Hyundai Elantra 2021-2023,378686
1,2,Hyundai Sonata 2022-2023,92795
2,3,Ford Mustang Mach-E 2021-2024,159114
3,4,Ford Mustang Mach-E 2021-2024,159114
4,5,Jaguar I-Pace 2018-2024,10182
...,...,...,...
1352,1353,MINI Mini 2001-2013,0
1353,1354,Audi A3 2006-2019,186313
1354,1355,Hyundai Accent III 2005-2010,0
1355,1356,Audi A3 2003-2012,48740
